In [1]:
import sys
sys.path.insert(1, '/home/taylor/Classes/cs230/achtung')
import achtung;

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [3]:
import gym
import numpy as np
import pickle

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecEnv
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import VecTransposeImage
# from stable_baselines3.common.atari_wrappers import AtariWrapper

In [5]:
env = achtung.Achtung()
env.speed = 0 # set to zero for training (i.e., no frame delay)
env.render_game = False

Achtung Die Kurve!


In [6]:
# for i in range(50):
#     action = 0
#     obs, rewards, done, info = env.step(action)
#     env.render()
    
#     plt.imshow(np.resize(obs, (achtung.OBS_HEIGHT, achtung.OBS_WIDTH)), cmap="gray") 
#     plt.show()

#     if done:
#         obs = env.reset()

In [7]:
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 3)
env = VecTransposeImage(env)

env.observation_space

Box(0, 255, (3, 80, 80), uint8)

In [8]:
model = PPO("CnnPolicy", 
            env, 
            n_steps=128,
            n_epochs=4,
            batch_size=256,
            learning_rate=2.5e-4,
            clip_range=0.1,
            clip_range_vf=0.1,
            vf_coef=0.5,
            ent_coef=0.01) 

In [9]:
# Evaluate the initial random policy
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=25)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:60.64 +/- 31.81


In [ ]:
# Train
rewards = []
stds = []

for i in range(100):
    print("iteration: ", i+1)
    model.learn(total_timesteps=10000)
    
    print("   saving...")
    model.save("ppo_achtung")
    
    print("   evalute...")
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic = True)
    print(f"   mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")
    
    rewards.append(mean_reward)
    stds.append(std_reward)
    
    with open("baselines_training/ppo_reward.txt", "wb") as f:   
        pickle.dump(rewards, f)
    with open("baselines_training/ppo_std.txt", "wb") as f:   
        pickle.dump(stds, f)

iteration:  1
   saving...
   evalute...
   mean_reward:29.50 +/- 12.33
iteration:  2
   saving...
   evalute...
   mean_reward:35.10 +/- 7.92
iteration:  3
   saving...
   evalute...
   mean_reward:40.90 +/- 5.84
iteration:  4
   saving...
   evalute...
   mean_reward:34.50 +/- 11.32
iteration:  5
   saving...
   evalute...
   mean_reward:54.90 +/- 23.90
iteration:  6
   saving...
   evalute...
   mean_reward:26.00 +/- 14.16
iteration:  7
   saving...
   evalute...
   mean_reward:25.40 +/- 25.44
iteration:  8
   saving...
   evalute...
   mean_reward:34.50 +/- 10.32
iteration:  9
   saving...
   evalute...
   mean_reward:34.30 +/- 11.89
iteration:  10
   saving...
   evalute...
   mean_reward:33.00 +/- 13.42
iteration:  11
   saving...
   evalute...
   mean_reward:40.20 +/- 23.01
iteration:  12
   saving...
   evalute...
   mean_reward:31.70 +/- 13.31
iteration:  13
   saving...
   evalute...
   mean_reward:57.40 +/- 22.63
iteration:  14
   saving...
   evalute...
   mean_reward:33.30

In [ ]:
# Evaluate the trained policy
model.load("ppo_achtung")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic = True)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")